# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 20.09.2022

Мягкий дедлайн: 23:59MSK 04.10.2022

Жесткий дедлайн: 23:59MSK 10.10.2022

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [88]:
#!pip install widgetsnbextension

In [73]:
import torch
from torch import cuda
from torch import nn
import pandas as pd
import numpy as np
import random

In [2]:
%pip install wget

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m wget  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip -o data.txt.zip


Saved under data.txt (1).zip


In [64]:
df = pd.read_csv('data.txt.zip', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515340,2006,51.28467,45.88068,22.19582,-5.53319,-3.61835,-16.36914,2.12652,5.18160,-8.66890,...,4.81440,-3.75991,-30.92584,26.33968,-5.03390,21.86037,-142.29410,3.42901,-41.14721,-15.46052
515341,2006,49.87870,37.93125,18.65987,-3.63581,-27.75665,-18.52988,7.76108,3.56109,-2.50351,...,32.38589,-32.75535,-61.05473,56.65182,15.29965,95.88193,-10.63242,12.96552,92.11633,10.88815
515342,2006,45.12852,12.65758,-38.72018,8.80882,-29.29985,-2.28706,-18.40424,-22.28726,-4.52429,...,-18.73598,-71.15954,-123.98443,121.26989,10.89629,34.62409,-248.61020,-6.07171,53.96319,-8.09364
515343,2006,44.16614,32.38368,-3.34971,-2.49165,-19.59278,-18.67098,8.78428,4.02039,-12.01230,...,67.16763,282.77624,-4.63677,144.00125,21.62652,-29.72432,71.47198,20.32240,14.83107,39.74909


In [78]:
!pip3 install pytorch 

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Крот\AppData\Local\Temp\pip-install-9sl6xfpf\pytorch_229763923b7d4d3db9b347f0f9d009c2\setup.py", line 15, in <module>
      raise Exception(message)
  Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  Running setup.py install for pytorch did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Крот\AppData\Local\Temp

In [79]:
!pip3 install pytorch torchvision cudatoolkit=10.1 -c pytorch

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [74]:
device = torch.cuda.device('cuda')

In [75]:
print(torch.cuda_version)

None


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

#s = np.std(y)
#m = np.mean(y)
#y = (y-m)/s

train_size = 463715
ts = 200000
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]
#X_train = X[:ts, :]
#y_train = y[:ts]
#X_test = X[ts:2*ts, :]
#y_test = y[ts:2*ts:]

In [52]:
pd.set_option('display.max_rows', 90) 
years = pd.DataFrame(df[0].value_counts())
years = years.reset_index()
years = years.rename(columns={'index':'year',0:'amount'})
years

,year,amount
0,2007,39404
1,2006,37534
2,2005,34952
3,2008,34760
4,2009,31038
5,2004,29607
6,2003,27382
7,2002,23451
8,2001,21590
9,2000,19285


In [53]:
df[df[0]==2004]

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
12,2004,26.51957,-148.15762,-13.30095,-7.25851,17.22029,-21.99439,5.51947,3.48418,2.61738,...,23.80442,251.76360,18.81642,157.09656,-27.79449,-137.72740,115.28414,23.00230,-164.02536,51.54138
111,2004,46.55475,67.72894,11.58950,-1.09344,-16.79960,3.42247,-24.16656,-0.06942,12.12003,...,7.33502,-114.31487,-160.43993,80.47994,1.40852,-2.76622,92.44024,5.10762,-1.87430,-6.39645
139,2004,48.22872,58.99264,9.97885,-4.63135,-34.57472,5.22466,1.40945,9.60903,-10.89927,...,6.66140,-141.29552,-122.32486,26.89910,14.42937,77.48995,14.11111,-0.09896,60.32697,-5.83586
140,2004,45.97266,40.30498,14.02324,7.29785,-2.34973,14.03147,-6.67732,0.02808,-17.55702,...,14.19458,-226.86980,134.11533,-16.84217,-45.31871,294.89540,33.63607,32.87204,-37.14914,-13.49010
141,2004,46.47517,43.43231,6.61022,14.36423,-4.88513,16.03377,1.59485,8.83188,-1.83059,...,9.33450,-107.94091,-173.59092,116.67985,8.36760,50.84033,340.77023,2.28266,-90.83755,2.99857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515089,2004,42.75115,40.64368,-44.08613,11.63948,2.46718,-3.59763,5.86888,-2.80991,-3.03441,...,44.14458,-584.59376,-35.94937,139.03100,1.06249,206.39111,140.44370,6.72757,292.19965,17.98996
515123,2004,48.60275,-10.58098,9.63721,-13.29541,-22.55836,-22.42514,0.83743,1.20253,-13.91216,...,40.33928,565.07063,201.29056,1.24398,15.04846,-399.68796,-15.98505,-18.37700,50.69444,-3.45966
515124,2004,47.23873,-14.10980,-5.78195,-8.01924,-27.14964,-15.35870,1.79275,-4.21822,-7.71629,...,-16.55767,159.32513,126.76655,-12.49798,3.75477,67.41327,-67.58944,13.18648,513.68723,22.13152
515125,2004,47.84952,17.52292,-4.16613,-16.62424,-24.12809,-22.66279,-4.92801,1.57621,-10.12602,...,30.84685,152.11086,143.01008,93.16518,12.35550,-24.43481,27.47237,-0.82630,49.99633,21.19850


In [54]:
int(years[years['year']==2004]['amount'])

29607

In [59]:
y = years['year'].to_numpy()
n = years['amount'].to_numpy() 
data = df
for year in y[n<15000]:
    N_b = 25000 +np.random.randint(1,10000)
    k = int(np.round(N_b/int(years[years['year']==year]['amount']))-1)
    data_app = pd.concat([df[df[0]==year]]*k,ignore_index=True)
    data = pd.concat([data,data_app],ignore_index=True)

In [61]:
years_new = pd.DataFrame(data[0].value_counts())
len(years_new)

89

In [62]:
from sklearn.utils import shuffle
data = shuffle(data)

In [63]:
data

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
905483,1981,46.80788,18.31238,48.55201,0.34229,-2.91631,-9.29472,-2.25177,-7.87464,8.95322,...,-8.17101,-69.33443,13.07657,-12.81587,-1.13975,38.43130,-98.39106,9.54846,90.55943,-0.32626
395534,2007,48.20859,28.87616,-12.44259,-9.22333,1.18085,-10.17362,2.41734,2.55788,-4.12803,...,3.76557,-127.18839,41.02659,-7.76034,13.47527,100.14590,-36.83506,3.13008,26.31209,2.86023
1591936,1961,32.50774,15.10591,-60.24619,-22.90413,16.33155,-13.30959,-13.59257,9.78554,-15.62065,...,-48.13925,-293.77596,363.52311,13.63275,-58.24210,10.92733,-119.13845,25.51992,-238.11619,-23.21972
2308251,1936,33.44969,-120.43485,71.94785,18.42320,-15.67761,24.87766,-32.51140,16.84437,-12.18785,...,113.86280,-47.89485,284.28184,-129.91011,-4.87341,-122.22951,765.66974,-14.91446,1492.40337,85.83563
186425,2004,41.42934,4.72901,-3.62541,-9.86392,-17.24566,-14.97305,17.37858,0.40546,-0.93322,...,-17.26900,-147.57268,36.80008,53.60872,-3.23467,42.24495,9.37914,2.27718,12.58872,-13.26597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386923,1938,37.08558,-143.92439,33.19571,3.90540,-55.78318,27.57952,-37.84551,2.56757,31.43338,...,48.81754,-157.30777,-47.32460,252.73581,-10.32529,-175.48375,45.71826,0.17633,149.15779,21.65891
1988819,1948,34.43976,-66.85256,19.23801,-5.61000,-9.75475,-3.11597,-7.51483,-0.71380,16.84672,...,35.80150,-33.12390,241.09570,2.60305,0.18348,-32.63681,266.80650,1.64543,264.34382,30.82098
1413665,1963,32.20105,-93.12566,-31.43473,-29.08274,-5.45235,20.75723,-18.21488,4.42776,-2.66908,...,12.28156,-105.45746,-324.93219,-23.73637,9.11744,53.38974,-152.87459,-44.79456,23.66246,-3.66018
743391,1988,49.64403,57.11516,-11.93402,15.69606,13.59054,-20.40071,-8.77495,-5.31301,11.74645,...,9.36555,-10.02302,180.71587,-53.67162,-16.41829,24.45099,-86.37880,7.52165,-41.79242,0.44690


In [ ]:
r = pd.DataFrame(X_train)
r.dtypes.unique()

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
reg1 = Ridge()
reg1.fit(X_train,y_train)
y_pred = reg1.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
RMSE

In [ ]:
len(np.unique(y_train))

In [ ]:
y_train[:]

## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [ ]:
from IPython.display import clear_output

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_norm = scaler.fit_transform(X_train)
x_test_norm = scaler.fit_transform(X_test)

In [ ]:
class RandomDataset(torch.utils.data.Dataset):
    """Our random dataset"""
    
    def __init__(self, x, y):
        self.x=x
        self.y=y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}

In [ ]:
x_norm_1 = torch.from_numpy(x_norm)
y_train = y_train.astype(np.single)
train_set = RandomDataset(x_norm_1,y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=512, shuffle=True)

x_test_norm = x_test_norm.astype(np.single)
y_test = y_test.astype(np.single)
test_set = RandomDataset(x_test_norm,y_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=True)



In [ ]:
import pdb

In [111]:
def train(model, optimizer, criterion, train_loader, test_loader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    epoch = 0
    val_loss = []
    val_epoch = []
    loss_ar = []
    val_accuracy = []
    rmse = []
    for epoch in range(0,epoches): # всего у нас будет 10 эпох (10 раз подряд пройдемся по всем батчам из трейна)
        print('Эпоха:', epoch)
        for data in tqdm(train_loader):
            samp = data['sample'] # берем батч из трейн лоадера
            y_pred = model(samp) # делаем предсказания
            targ = data['target']
            loss = criterion(y_pred, targ)
            loss.backward() # считаем градиенты обратным проходом
            optimizer.step() # обновляем параметры сети
            optimizer.zero_grad()
        if flag == 1:
            with torch.no_grad():
                t1 = torch.empty((0,1))
                t2 = torch.empty((0,1))# мы считаем качество, поэтому мы запрещаем фреймворку считать градиенты по параметрам
                for test in tqdm(test_loader):
                    x_val = data['sample']
                    y_pred = model(x_val)
                    y_val = data['target']
                    loss = criterion(y_pred, y_val) 
                    loss_ar.append(loss)
                    y_val = y_val.view_as(y_pred)
                    t1 = torch.cat((t1,y_pred),0)
                    t2 = torch.cat((t2,y_val),0)
                    #print('shape y_pred:', y_pred.shape,'\n','shape y_val:',y_val.shape)
                print(t1.shape)
                print(t2.shape)
                mse = mean_squared_error(t2.numpy(),t1.numpy())
                rmse.append(np.sqrt(mean_squared_error(t2,t1)))
                t1 = torch.round(t1)
                t = np.array((t1 == t2).numpy().tolist())
                val_accuracy.append(np.mean(t))
                val_loss.append(np.mean(loss_ar))
                val_epoch.append(epoch)
                clear_output(True)
                fig = plt.figure(figsize=(16,20))
                ax1,ax2,ax3 = fig.subplots(3,1)
                ax1.plot(val_epoch, val_loss)
                ax1.grid()
                ax1.set_title('Loss')
                ax2.plot(val_epoch,val_accuracy,color = 'orange')
                ax2.grid()
                ax2.set_title('Accuracy')
                ax3.plot(val_epoch,rmse,color='red')
                ax3.grid()
                ax3.set_title('RMSE')
                plt.show()

def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    predicts = torch.ones(len(test_loader.dataset))
    pred = torch.empty((0,1))
    y_te = torch.empty((0,1))
    with torch.no_grad():
        for data in tqdm(test_loader): # берем батч из трейн лоадера
            #pdb.set_trace()
            y_pred = model(data['sample'])
            # делаем предсказания
            loss = criterion(y_pred, data['target']) 
            #print('y_pred shape:', y_pred.shape)
            #print('pred shape:', data['target'].shape)
            pred = torch.cat((pred,y_pred),0)
            y_te = torch.cat((y_te,data['target'].view_as(y_pred)),0)
            #print(pred)
            #print("Пред:",pred)
            #print("Пред shape:",pred.shape)
            #print("Предикт:",predicts)
            #print("Предикт shape:",predicts.shape)
    return pred,y_te

In [136]:
def train(model, optimizer, criterion, train_loader, test_loader):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    epoch = 0
    val_loss = []
    val_epoch = []
    loss_ar = []
    val_accuracy = []
    rmse = []
    for epoch in range(0,epoches): # всего у нас будет 10 эпох (10 раз подряд пройдемся по всем батчам из трейна)
        #print('Эпоха:', epoch)
        for data in train_loader:
            samp = data['sample'] # делаем предсказания
            y_pred = model(samp)*std_target+mean_target      
            targ = data['target']*std_target+mean_target
            #print("min=", torch.min(targ),"max=",torch.max(targ))
            loss = criterion(y_pred, targ)
            loss.backward() # считаем градиенты обратным проходом
            optimizer.step() # обновляем параметры сети
            optimizer.zero_grad()
        if flag == 1:
            with torch.no_grad():
                t1 = torch.empty((0,1))
                t2 = torch.empty((0,1))# мы считаем качество, поэтому мы запрещаем фреймворку считать градиенты по параметрам
                for test in tqdm(test_loader):
                    x_val = data['sample']
                    y_pred = model(x_val)*std_target+mean_target      
                    y_val = data['target']
                    print("min=", torch.min(y_val),"max=",torchch.max(y_val))
                    y_val = y_val*std_target+mean_target
                    loss = criterion(y_pred, y_val) 
                    loss_ar.append(loss)
                    y_val = y_val.view_as(y_pred)
                    t1 = torch.cat((t1,y_pred),0)
                    t2 = torch.cat((t2,y_val),0)
                    #print('shape y_pred:', y_pred.shape,'\n','shape y_val:',y_val.shape)
                print(t1.shape)
                print(t2.shape)
                mse = mean_squared_error(t2.numpy(),t1.numpy())
                rmse.append(np.sqrt(mean_squared_error(t2,t1)))
                t1 = torch.round(t1)
                t = np.array((t1 == t2).numpy().tolist())
                val_accuracy.append(np.mean(t))
                val_loss.append(np.mean(loss_ar))
                val_epoch.append(epoch)
                clear_output(True)
                fig = plt.figure(figsize=(16,20))
                ax1,ax2,ax3 = fig.subplots(3,1)
                ax1.plot(val_epoch, val_loss)
                ax1.grid()
                ax1.set_title('Loss')
                ax2.plot(val_epoch,val_accuracy,color = 'orange')
                ax2.grid()
                ax2.set_title('Accuracy')
                ax3.plot(val_epoch,rmse,color='red')
                ax3.grid()
                ax3.set_title('RMSE')
                plt.show()

def test(model, criterion, test_loader):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    predicts = torch.ones(len(test_loader.dataset))
    pred = torch.empty((0,1))
    y_te = torch.empty((0,1))
    with torch.no_grad():
        for data in test_loader: # берем батч из трейн лоадера
            #pdb.set_trace()
            y_pred = model(data['sample'])
            # делаем предсказания
            loss = criterion(y_pred, data['target']) 
            #print('y_pred shape:', y_pred.shape)
            #print('pred shape:', data['target'].shape)
            pred = torch.cat((pred,y_pred),0)
            y_te = torch.cat((y_te,data['target'].view_as(y_pred)),0)
            #print(pred)
            #print("Пред:",pred)
            #print("Пред shape:",pred.shape)
            #print("Предикт:",predicts)
            #print("Предикт shape:",predicts.shape)
    return pred,y_te

In [ ]:
lr = 0.01
n_neirons_1 = 1024
batch = 70
epoches = 3
flag = 1

In [ ]:
model = nn.Sequential()                 # создаем пустую модель, в которую будем добавлять слои
model.add_module('l1', nn.Linear(90, n_neirons_1)) # добавили слой с 5-ю нейронами на вход и 3-мя на выход
model.add_module('l2', nn.Sigmoid())       # добавили функцию активации
#model.add_module('l3', nn.Linear(n_neirons_1, 258))
#model.add_module('l4', nn.ReLU()) 
#model.add_module('l5', nn.Linear(258, 10))
#model.add_module('l6', nn.Sigmoid()) 
model.add_module('l7', nn.Linear(n_neirons_1, 1))
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
flag = 0
epoches = 5
for n in [32,64,90,128,256,512,712,1024,1250,1500]:
    for lr in [0.001,0.01,0.1]:
        model = nn.Sequential()                 # создаем пустую модель, в которую будем добавлять слои
        model.add_module('l1', nn.Linear(90, n)) # добавили слой с 5-ю нейронами на вход и 3-мя на выход
        model.add_module('l2', nn.Sigmoid())       # добавили функцию активации
        #model.add_module('l3', nn.Linear(n_neirons_1, 258))
        #model.add_module('l4', nn.ReLU()) 
        #model.add_module('l5', nn.Linear(258, 10))
        #model.add_module('l6', nn.Sigmoid()) 
        model.add_module('l7', nn.Linear(n, 1))
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        train(model,optimizer,criterion,train_loader,test_loader)
        y_pred,y_te =  test(model, criterion, test_loader)
        RMSE_1 = np.sqrt(mean_squared_error(y_te,y_pred))
        print("Число нейронов = ", n,"Lern rate=", lr, "RMSE= ",RMSE_1,'\n','predict[:10]=',y_pred[:20],'\n')

In [ ]:
lr = 0.1
for n in [512,712,1024,1250,1500]:
    for epoches in range(3,15):
        model = nn.Sequential()                 # создаем пустую модель, в которую будем добавлять слои
        model.add_module('l1', nn.Linear(90, n)) # добавили слой с 5-ю нейронами на вход и 3-мя на выход
        model.add_module('l2', nn.Sigmoid())       # добавили функцию активации
        #model.add_module('l3', nn.Linear(n_neirons_1, 258))
        #model.add_module('l4', nn.ReLU()) 
        model.add_module('l5', nn.Linear(n, 90))
        model.add_module('l6', nn.Tanh()) 
        model.add_module('l7', nn.Linear(90, 1))
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        train(model,optimizer,criterion,train_loader,test_loader)
        y_pred,y_te =  test(model, criterion, test_loader)
        RMSE_1 = np.sqrt(mean_squared_error(y_te,y_pred))
        print("Число нейронов = ", n,"Эпох = ",epoches , "RMSE= ",RMSE_1,'\n','predict and test=',torch.cat((y_pred[:10],y_te[:10]),1),'\n')

In [ ]:
torch.min(y_pred)

In [ ]:
torch.max(y_pred)

In [ ]:
assert test(model, criterion, test_loader).shape[0] == y_test.shape[0]

In [ ]:
y_pred,y_te =  test(model, criterion, test_loader)
RMSE_1 = np.sqrt(mean_squared_error(y_te*s+m,y_pred*s+m))
RMSE_1

In [ ]:
train(model,optimizer,criterion,train_loader,test_loader)

In [ ]:
y_pred =  test(model, criterion, test_loader)
RMSE_1 = np.sqrt(mean_squared_error(y_test*s+m,y_pred*s+m))
RMSE_1

In [137]:
data_l = df.sample(10000)
X_l = data_l.iloc[:, 1:].values
y_l = data_l.iloc[:, 0].values
std_target = np.std(y_l)
mean_target = np.mean(y_l)
y_l = (y_l-mean_target)/std_target
l_size = 8000
X_train_l = X_l[:l_size, :]
y_train_l = y_l[:l_size]
X_test_l = X_l[l_size:, :]
y_test_l = y_l[l_size:]
x_norm_l = scaler.fit_transform(X_train_l)
x_test_norm_l = scaler.fit_transform(X_test_l)

In [138]:
x_norm_l = torch.from_numpy(x_norm_l)
y_train_l = y_train_l.astype(np.single)
train_set_l = RandomDataset(x_norm_l,y_train_l)
train_loader_l = torch.utils.data.DataLoader(train_set_l, batch_size=2)

x_test_norm_l = x_test_norm_l.astype(np.single)
y_test_l = y_test_l.astype(np.single)
test_set_l = RandomDataset(x_test_norm_l,y_test_l)
test_loader_l = torch.utils.data.DataLoader(test_set_l, batch_size=1000)

In [139]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [140]:
criterion_l = RMSELoss()

In [142]:
lr = 0.1
epoches = 8
for n1 in [32,64,90,128,256]:
    for n2 in [512,728,1024,1500]:
        for lr in [0.005,0.01,0.001]:
            model_l = nn.Sequential()                 # создаем пустую модель, в которую будем добавлять слои
            model_l.add_module('l1', nn.Linear(90, 256)) # добавили слой с 5-ю нейронами на вход и 3-мя на выход
            model_l.add_module('l2', nn.Sigmoid())       # добавили функцию активации
            model_l.add_module('l3', nn.Linear(256, 1))
            #model_l.add_module('l4', nn.ReLU()) 
            #model_l.add_module('l5', nn.Linear(n1, n2))
            #model_l.add_module('l6', nn.Tanh()) 
            #model_l.add_module('l7', nn.Linear(n2, 90))
            #model_l.add_module('l8', nn.Sigmoid())
            #model_l.add_module('l9', nn.Linear(90, 1))
            optimizer_l = torch.optim.AdamW(model_l.parameters(), lr=lr)
            train(model_l,optimizer_l,criterion_l,train_loader_l,test_loader_l)
            y_pred_l,y_te_l =  test(model_l, criterion_l, test_loader_l)
            y_pred_l = y_pred_l*std_target+mean_target
            y_te_l = y_te_l*std_target+mean_target
            RMSE_1_l = np.sqrt(mean_squared_error(y_te_l,y_pred_l))
            print("Число нейронов 1 = ", n1,"Число нейронов 2 = ", n2,"Lr = ",lr , "RMSE= ",RMSE_1,
                  "min=",torch.min(y_pred_l),"max=",torch.max(y_pred_l),'\n',
                  'predict and test=',torch.cat((y_pred_l[:10],y_te_l[:10]),1),'\n')

C:\Users\72CD~1\AppData\Local\Temp/ipykernel_4808/1551483959.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broa

Число нейронов 1 =  32 Число нейронов 2 =  512 Lr =  0.005 RMSE=  10.88653 min= tensor(2000.6686) max= tensor(2001.0930) 
 predict and test= tensor([[2000.6926, 2001.0000],
        [2000.6821, 1975.0000],
        [2000.6890, 1999.0000],
        [2000.6975, 2005.0000],
        [2000.6829, 2005.0000],
        [2000.6866, 1979.0000],
        [2000.6807, 2001.0000],
        [2000.6837, 1995.0000],
        [2000.6880, 1995.0000],
        [2000.6846, 2006.0000]]) 



c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\72CD~1\AppData\Local\Temp/ipykernel_4808/1551483959.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broa

Число нейронов 1 =  32 Число нейронов 2 =  512 Lr =  0.01 RMSE=  10.88653 min= tensor(2000.9318) max= tensor(2000.9319) 
 predict and test= tensor([[2000.9318, 2001.0000],
        [2000.9318, 1975.0000],
        [2000.9318, 1999.0000],
        [2000.9318, 2005.0000],
        [2000.9318, 2005.0000],
        [2000.9318, 1979.0000],
        [2000.9318, 2001.0000],
        [2000.9318, 1995.0000],
        [2000.9318, 1995.0000],
        [2000.9318, 2006.0000]]) 



c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\72CD~1\AppData\Local\Temp/ipykernel_4808/1551483959.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broa

...

c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\72CD~1\AppData\Local\Temp/ipykernel_4808/1551483959.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broa

Число нейронов 1 =  32 Число нейронов 2 =  1024 Lr =  0.01 RMSE=  10.88653 min= tensor(2000.9318) max= tensor(2000.9319) 
 predict and test= tensor([[2000.9318, 2001.0000],
        [2000.9318, 1975.0000],
        [2000.9318, 1999.0000],
        [2000.9318, 2005.0000],
        [2000.9318, 2005.0000],
        [2000.9318, 1979.0000],
        [2000.9318, 2001.0000],
        [2000.9318, 1995.0000],
        [2000.9318, 1995.0000],
        [2000.9318, 2006.0000]]) 



c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\72CD~1\AppData\Local\Temp/ipykernel_4808/1551483959.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {'sample': torch.tensor(self.x[idx, :], dtype=torch.float), 'target': self.y[idx]}
c:\users\крот\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broa

Число нейронов 1 =  32 Число нейронов 2 =  1024 Lr =  0.001 RMSE=  10.88653 min= tensor(1999.0153) max= tensor(2002.5807) 
 predict and test= tensor([[2000.4320, 2001.0000],
        [1999.9941, 1975.0000],
        [2000.2166, 1999.0000],
        [2000.5077, 2005.0000],
        [1999.8027, 2005.0000],
        [2000.0837, 1979.0000],
        [1999.9772, 2001.0000],
        [2000.1675, 1995.0000],
        [2000.2307, 1995.0000],
        [1999.9829, 2006.0000]]) 



KeyboardInterrupt: 

# Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

In [ ]:
# YOUR CODE HERE (－.－)...zzzZZZzzzZZZ